In [1]:
import numpy as np
from scipy.optimize import least_squares
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
#
# if pade.py is not in the current directory, set this path:
#
import sys
sys.path.append('../Python_libs')
import rac_pade_functions as racx

In [2]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#  files in the current directory do not need the path name
#
#df = pd.read_csv("/home/thomas/Python/StabPlots/Stab_data/1D_a0.2_b0_c0.14/crossing_1.dat", delim_whitespace=True)
df = pd.read_csv("sb_rac.csv")
#df = pd.read_csv("crossing_1.dat", delim_whitespace=True)
plt.cla()
plt.plot(df.l.values, df.E1.values, 'o-')
plt.plot(df.l.values, df.E2.values, 'o-')
plt.plot(df.l.values, df.E3.values, 'o-')
plt.show()
df[:5]

,l,E1,E2,E3,E4
0,0.0,0.469219,1.365173,1.938043,3.271038
1,0.2,0.457846,1.314401,1.844238,3.242423
2,0.4,0.446073,1.251127,1.760926,3.215267
3,0.6,0.433754,1.173608,1.690292,3.189317
4,0.8,0.420648,1.082215,1.632448,3.164365


In [3]:
i_neg = np.argmin(abs(df.E1.values))
if df.E1[i_neg] > 0:
    i_neg += 1
ls = df.l.values[i_neg:]
print('N=',len(ls))
Es = df.E1.values[i_neg:]
plt.cla()
plt.plot(df.l.values, df.E1.values, 'b-')
plt.plot(df.l.values, df.E2.values, 'b-')
plt.plot(df.l.values, df.E3.values, 'b-')
plt.plot(ls, Es, 'o-', color="orange")
plt.show()

N= 89


In [11]:
k2s = -Es
ks  = np.sqrt(k2s)

In [4]:
def pade_31_lsq(params, k, ksq, lbs):
    """ 
    rac31 - lambda_i for least_squares 
    """
    l = params[0]
    a = params[1]
    b = params[2]
    d = params[3]
    a4b2=a*a*a*a + b*b
    aak2=a*a*k*2
    ddk=d*d*k
    num = (ksq + aak2 + a4b2) * (1 + ddk)
    den = a4b2 + aak2 + ddk*a4b2
    rac31 = l * num / den
    return rac31 - lbs

In [5]:
def pade_31j_lsq(params, k, ksq, lbs):
    """
    see RAC-31_derivatives notebook for formulas
    least_squares() needs the transpose 
    """
    l = params[0]
    a = params[1]
    b = params[2]
    d = params[3]
    a2 = a*a
    b2 = b*b
    d2 = d*d
    a4b2 = a2*a2 + b2
    aak2 = a2*k*2
    ddk = d2*k
    fr1 = (ksq + aak2 + a4b2)
    fr2 = (1 + ddk)
    den = a4b2 + aak2 + ddk*a4b2
    dl = fr1*fr2/den
    #f = l*dl
    da = -4*a*ksq*l * fr2 * (a2*a2*d2 + a2*fr2 - b2*d2 + k) / den**2
    db = -2*b*ksq*l * fr2 * (2*a2*d2 + fr2) / den**2
    dd = 4*a2*d*ksq*l * fr1/den**2
    return np.transpose(np.array([dl, da, db, dd]))

In [6]:
E0, G0 = 2.00, 2/7
p0s=[1.1] + racx.guess(E0, G0) + [0.5]

In [12]:
#
#  the derivative of [3,1] works
#
#pade_31(ks, k2s, p0s)
#df1s = pade_31_num_jac(ks[87:], k2s[87:], p0s, step=1e-2, tiny=1e-4)
#print("num grad:", df1s)

df2s = pade_31j_lsq(p0s, ks[-8:], k2s[-8:], ls)
print("ana grad:", df2s)

ana grad: [[ 6.96604072 -4.45429151 -8.45700969  1.26297088]
 [ 7.04153274 -4.52893247 -8.56148799  1.28497313]
 [ 7.11705322 -4.60382536 -8.66597487  1.30708472]
 [ 7.1926019  -4.67896701 -8.77047038  1.32930464]
 [ 7.26817855 -4.75435432 -8.87497459  1.35163191]
 [ 7.34378293 -4.82998426 -8.97948757  1.37406556]
 [ 7.4194148  -4.90585387 -9.08400936  1.39660463]
 [ 7.49507394 -4.98196024 -9.18854002  1.41924818]]


In [13]:
E0, G0 = 2.00, 2/7
p0s=[1.1] + racx.guess(E0, G0) + [0.5]

In [14]:
#
#  test results of other minimizers wrt least squares
#
#p0s=res.x

print("Start parameters:",p0s)
print('Least squares, 2-point jac')
res = least_squares(pade_31_lsq, p0s, jac='2-point', args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Start parameters: [1.1, 0.22466750413864453, 1.4151140501139234, 0.5]
Least squares, 2-point jac
njev: 9
cost: 0.00013419409088779363
grad: [-5.93237629e-11 -4.41563465e-09 -3.44036373e-09 -1.67383956e-10]
message: `gtol` termination condition is satisfied.
success: True
x: [2.40228618 0.27139797 1.28131923 0.45438948]
chi2 = 3.016e-06
Er=1.636354,  Gamma=0.377512


In [21]:
print('Least squares, trust-region-reflective (default) with analytic jac')
res = least_squares(pade_31_lsq, p0s, method='trf', jac=pade_31j_lsq, args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Least squares, analytic jac
njev: 9
cost: 0.0001341940908877723
grad: [ 5.01070595e-10  2.17635177e-10 -1.58871055e-09  1.98734293e-09]
message: `gtol` termination condition is satisfied.
success: True
x: [2.40228618 0.27139798 1.28131923 0.4543895 ]
chi2 = 3.016e-06
Er=1.636354,  Gamma=0.377512


In [22]:
print('Least squares, Levenberg-Marquardt with analytic jac')
res = least_squares(pade_31_lsq, p0s, method='lm', jac=pade_31j_lsq, args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Least squares, Levenberg-Marquardt with analytic jac
njev: 7
cost: 0.00013419409088795729
grad: [ 3.08132647e-09 -3.10557394e-09 -8.82546995e-09  6.05306466e-09]
message: `ftol` termination condition is satisfied.
success: True
x: [2.40228618 0.271398   1.28131922 0.45438954]
chi2 = 3.016e-06
Er=1.636354,  Gamma=0.377512


In [23]:
print('Least squares, TRF with bounds')

bnds=([0,0,0,0],[np.inf,np.inf,np.inf,np.inf])

res = least_squares(pade_31_lsq, p0s, jac=pade_31j_lsq, bounds=bnds, args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Least squares, TRF with bounds
njev: 31
cost: 0.0001341940908877327
grad: [ 2.06678265e-10  3.65353220e-10 -7.13859853e-10  1.20229661e-09]
message: `gtol` termination condition is satisfied.
success: True
x: [2.40228618 0.27139798 1.28131923 0.45438949]
chi2 = 3.016e-06
Er=1.636354,  Gamma=0.377512
